In [1]:
from onnxruntime.quantization.quantize import quantize_dynamic
from segment_anything.utils.onnx import SamOnnxModel
from onnxruntime.quantization import QuantType
from segment_anything import SamPredictor
from segment_anything import build_sam
import onnxruntime
import numpy as np
import torch
import cv2

/opt/conda/envs/torch/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/envs/torch/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/opt/conda/envs/torch/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZNK3c107SymBool10guard_boolEPKcl'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
model        = build_sam('checkpoint/sam_vit_h.pth')
onnx_model   = SamOnnxModel(model = model, return_single_mask = True)
dynamic_axes = {
                    'point_coords' : {1 : 'num_points'},
                    'point_labels' : {1 : 'num_points'}
              
                }

embed_dim  = model.prompt_encoder.embed_dim
embed_size = model.prompt_encoder.image_embedding_size

mask_input_size = [4 * x for x in embed_size]
dummy_inputs    = {
                        'image_embeddings' : torch.randn(1, embed_dim, *embed_size, dtype = torch.float),
                        'point_coords'     : torch.randint(low = 0, high = 1024, size = (1, 5, 2), dtype = torch.float),
                        'point_labels'     : torch.randint(low = 0, high = 4, size = (1, 5), dtype = torch.float),
                        'mask_input'       : torch.randn(1, 1, *mask_input_size, dtype = torch.float),
                        'has_mask_input'   : torch.tensor([1], dtype=torch.float),
                        'orig_im_size'     : torch.tensor([1500, 2250])
                    }

_ = onnx_model(**dummy_inputs)
output_names = ['masks', 'iou_predictions', 'low_res_masks']

In [3]:
onnx_model_path  = 'checkpoint/onnx_sam.onnx'
quant_model_path = 'checkpoint/quant_sam.onnx' 

with open('checkpoint/onnx_sam.onnx', 'wb') as f:
    
    torch.onnx.export(
                        onnx_model, tuple(dummy_inputs.values()),
                        f, export_params = True, verbose = False, opset_version = 16,
                        do_constant_folding = True, input_names = list(dummy_inputs.keys()),
                        output_names = output_names, dynamic_axes = dynamic_axes
                    )

/opt/conda/envs/torch/lib/python3.10/site-packages/segment_anything/modeling/transformer.py:210: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  x = x.reshape(b, n, num_heads, c // num_heads)
/opt/conda/envs/torch/lib/python3.10/site-packages/segment_anything/modeling/transformer.py:232: TracerWarning: Converting a tensor to a Python float might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  attn = attn / math.sqrt(c_per_head)
/opt/conda/envs/torch/lib/python3.10/site-packages/segment_anything/utils/onnx.p

In [4]:
quantize_dynamic(
    model_input    = onnx_model_path,
    model_output   = quant_model_path,
    optimize_model = True,
    per_channel    = False, 
    reduce_range   = False,
    weight_type    = QuantType.QUInt8
)

onnx_model_path = quant_model_path
onnx_model_path

Ignore MatMul due to non constant B: /[MatMul_310]
Ignore MatMul due to non constant B: /[MatMul_314]
Ignore MatMul due to non constant B: /[MatMul_424]
Ignore MatMul due to non constant B: /[MatMul_428]
Ignore MatMul due to non constant B: /[MatMul_556]
Ignore MatMul due to non constant B: /[MatMul_560]
Ignore MatMul due to non constant B: /[MatMul_670]
Ignore MatMul due to non constant B: /[MatMul_674]
Ignore MatMul due to non constant B: /[MatMul_785]
Ignore MatMul due to non constant B: /[MatMul_789]
Ignore MatMul due to non constant B: /[MatMul_917]
Ignore MatMul due to non constant B: /[MatMul_921]
Ignore MatMul due to non constant B: /[MatMul_1032]
Ignore MatMul due to non constant B: /[MatMul_1036]
Ignore MatMul due to non constant B: /[MatMul_1179]


'checkpoint/quant_sam.onnx'

In [5]:
ort_session = onnxruntime.InferenceSession(onnx_model_path)

model.to(device = 'cuda')
predictor = SamPredictor(model)

AssertionError: Torch not compiled with CUDA enabled

In [ ]:
image = cv2.cvtColor(cv2.imread('checkpoint/src_image.jpg'), cv2.COLOR_BGR2RGB)
predictor.set_image(image)

In [ ]:
image_embedding = predictor.get_image_embedding().cpu().numpy()
image_embedding.shape

In [ ]:
input_point = np.array([[500, 375]])
input_label = np.array([1])

onnx_coord  = np.concatenate([input_point, np.array([[0.0, 0.0]])], axis = 0)[None, :, :]
onnx_label  = np.concatenate([input_label, np.array([-1])], axis = 0)[None, :].astype(np.float32)
onnx_coord  = predictor.transform.apply_coords(onnx_coord, image.shape[:2]).astype(np.float32)

onnx_mask_input     = np.zeros((1, 1, 256, 256), dtype = np.float32)
onnx_has_mask_input = np.zeros(1, dtype = np.float32) 